# 2.2 Negation as Failure

Negation as Failure (NAF) searches for an absence of data - inferring new facts where a pattern is missing.

There are two critical components to understand when using NAF with Datalog rules, **NOT** and **NOT EXISTS**.

## NOT

When used alone, **NOT** checks for a pattern to be false.

### The Closed World Assumption

RDFox follows the closed world assumption, which assumes statements to be false if they cannot be proven true.

This means that the pattern can either be provably false or not present in the data at all to trigger the inference.

## NOT EXISTS

Subtly different from **NOT**, **NOT EXISTS** allows us to infer new facts based on missing data.

**NOT EXISTS** searches the data store for the described pattern. If the returned result set is empty, **NOT EXISTS** returns True and vice versa.

# Example

The following example highlights the different implications of **NOT** vs **NOT EXISTS** in the context of a vehicle sensor as it approaches a traffic light.

In [95]:
naf_data = """
@prefix : <https://rdfox.com/example#> .

:leftSensor a :Camera ;
    :hasReading :left-001 .

:rightSensor a :Camera ;
    :hasReading :right-001 .

:highSensor a :Camera ;
    :hasReading :high-001 ;
    :hasReading :high-002 .

:lowSensor a :Camera ;
    :hasReading :low-001 .



:left-001 a :SensorImage ;
    :trafficLightPresent :green .

:right-001 a :SensorImage ;
    :trafficLightPresent :red .

:high-001 a :SensorImage ;
    :trafficLightPresent :green .

:high-002 a :SensorImage ;
    :trafficLightPresent :red .

:low-001 a :SensorImage .

"""

In [100]:
naf_rules = """

[?camera, :message, "Sensor observed green light. "] :-
    [?camera, :hasReading, ?reading],
    NOT [?reading, :trafficLightPresent, :red] .

[?camera, :message, "Sensor observed no red light."] :-
    [?camera, a, :Camera] ,
    NOT EXISTS ?reading IN (
        [?camera, :hasReading, ?reading],
        [?reading, :trafficLightPresent, :red]
    ) .

"""

In [101]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=naf_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=naf_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue NOT select query
ft_sparql_text = "SELECT ?camera ?message ?colour WHERE { ?camera a :Camera ; :hasReading ?reading; :message ?message. OPTIONAL { SELECT ?reading ?colour WHERE { ?reading :trafficLightPresent ?colour . } } } ORDER BY ASC(?message) ASC(?camera)"
ft_response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": ft_sparql_text})
assert_response_ok(ft_response, "Failed to run ft select query.")
print('\n=== Messages based on objects in images ===')
print(ft_response.text)


=== Messages based on objects in images ===
?camera	?message	?colour
<https://rdfox.com/example#highSensor>	"Sensor observed green light. "	<https://rdfox.com/example#green>
<https://rdfox.com/example#highSensor>	"Sensor observed green light. "	<https://rdfox.com/example#red>
<https://rdfox.com/example#leftSensor>	"Sensor observed green light. "	<https://rdfox.com/example#green>
<https://rdfox.com/example#lowSensor>	"Sensor observed green light. "	
<https://rdfox.com/example#leftSensor>	"Sensor observed no red light."	<https://rdfox.com/example#green>
<https://rdfox.com/example#lowSensor>	"Sensor observed no red light."	



### Inference Table

| Lights present in image | Inferred by NOT | Inferred by NOT EXISTS |
|:---:|:---:|:---:|
| (Left)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;green | YES | YES |
| (Right)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;red | NO | NO |
| (High)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;both | YES | NO |
| (Low)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;none | YES | YES |

## Incremental retraction

NAF is an incredibly powerful tool and raises an potentially thorny question when combined with the Incremental Reasoning of RDFox.

What happens when new data is added that fills the gap of an absent triple, whose absence was leading to the inference of another fact?

Well, the previously inferred fact will have to be retracted.

Try importing some new data that contains something from the **NOT EXISTS** atom and see what happens.

## Cycles in rules

A rule, or set of rules, cannot be imported if they create cyclic logic, meaning an inferred triples change the conditions that led to their inference.

This is known as a stratification error - the rule set cannot be stratified (logically chained in a way that terminates).

### Where can cycles appear?

It can be easy to write rules that cannot be stratified when using Aggregation and Negation.

This behavior can be difficult to spot so RDFox will prevent you from importing such rules and will tell you where the cycle occurs.

With a simple example, it's easy to see. Take this rule, can you see the problem?

In [103]:
datalog_rule = """
prefix : <https://rdfox.com/example#>

[?x, a, :Class] :-
    [?x, a, ?c] ,
    NOT [?x, a, :Class] .
"""

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print(response)

Exception: Failed to add rule.
Status received=400
RuleCompilationException: The program is not stratified because these components of the dependency graph contain cycles through negation and/or aggregation:
======== COMPONENT 1 ========
    <https://rdfox.com/example#Class>[?x] :- rdf:type[?x, ?c], NOT <https://rdfox.com/example#Class>[?x] .
========================================================================================================================


## Where is NAF relevant?

Negation as failure is a particularly powerful tool as the absence of data is a common concept in real-world applications.

### Publishing

To generate advanced recommendations, offer detailed search, control user access, etc.

### Construction and Manufacturing

To determine complex compatibilities, comply with schematics and regulations, simulate failure conditions, etc.

### On-device

To encode real-world decision-making rules, map dependencies, repair data, etc.


## Exercise

Complete the rule `nafRules.dlog` in the `rules` folder so that the query below can be used to directly find...

In [ ]:
agg_sparql = """
SELECT ?starProduct ?averageStars ?reviewCount
WHERE {
    ?starProduct a :StarProduct ;
    :hasAverageStars ?averageStars ;
    :hasReviewCount ?reviewCount .
} ORDER BY DESC(?averageStars)
"""

Here is a representative sample of the data in `nafData.ttl`.

In [ ]:
sammple_data = """
@prefix : <https://rdfox.com/example#> .

:product0001 a :Sofa ;
    :hasReview :review11.

:review11 :hasStars 5 .

"""

In [3]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Read rules from external file
with open("../rules/2_2-NAF.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)
assert_response_ok(response, "Failed to add rule.")

print(response)

Results:  <Response [200]>


### Check your work

Run the query below to verify the results.

In [4]:
# Read query from external file
with open("../queries/star.rq", "r") as query_file:
    query = query_file.read()

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results:\n", queryResponse.text)

Results: 
 ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>



In [19]:
# Issue insert
sparql_insert = "prefix : <https://rdfox.com/example#> INSERT { ?s :hasProp 'Property' } WHERE { ?s :b ?o }"
response = requests.post(
    rdfox_server + "/datastores/default/sparql", data={"update": sparql_insert})
assert_response_ok(response, "Failed to insert fact via sparql.")

# Get response
queryResponse = requests.get(rdfox_server + "/datastores/default/sparql", params={"query": query})

# Catch errors
assert_response_ok(queryResponse, "Failed to run select query.")

print("Results: ", queryResponse.text)

Results:  ?s	?p	?o
<https://rdfox.com/example#a>	<https://rdfox.com/example#b>	<https://rdfox.com/example#c>
<https://rdfox.com/example#c>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#a>
<https://rdfox.com/example#a>	<https://rdfox.com/example#wooooow>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#excellent>	<https://rdfox.com/example#c>
<https://rdfox.com/example#a>	<https://rdfox.com/example#hasProp>	"Property"
<https://rdfox.com/example#a>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<https://rdfox.com/example#Class>



### You should see...

=== Star Products ===
|?starProduct|?averageStars|	?reviewCount|
|-----------|-------------|-------------|
|<https://rdfox.com/example/product0579>|	4.5|	8|
|<https://rdfox.com/example/product0934>|	4.5|	6|
|<https://rdfox.com/example/product0395>|	4.428571428571428571|	7|
|<https://rdfox.com/example/product0209>|	4.375|	8|
|<https://rdfox.com/example/product0317>|	4.285714285714285714|	7|
|<https://rdfox.com/example/product0222>|	4.166666666666666667|	6|
|<https://rdfox.com/example/product0931>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0492>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0137>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0326>|	4.125|	8|